In [2]:
import sys, traceback
import nest_asyncio
nest_asyncio.apply()

In [3]:
endpoint = "PLEASE_ENTER_YOUR_OWN_AZURE_COSMOS_GREMLIN_DB_NAME.gremlin.cosmosdb.azure.com"
graph_database = "recipes-database"
graph_collection = "recipes-graph"
primary_access_key = "PLEASE_ENTER_YOUR_OWN_AZURE_COSMOS_GREMLIN_DB_PRIMARY_ACCESS_KEY"

In [4]:
from gremlin_python.driver import client, serializer

client_connection = client.Client(
    "wss://" + endpoint + ":443/", "g",
    username="/dbs/" + graph_database + "/colls/" + graph_collection,
    password=primary_access_key,
    message_serializer=serializer.GraphSONSerializersV2d0()
)

In [5]:
from gremlin_python.driver.protocol import GremlinServerError

cosmosdb_messages = {
    409: 'Conflict exception. You\'re probably inserting the same ID again.',
    429: 'Not enough RUs for this query. Try again.'
}

def executeGremlinQuery(gremlinQuery, message=None, params=None):
    try: 
        callback = client_connection.submitAsync(gremlinQuery)
        if callback.result() is not None:
            return callback.result().one()
    except GremlinServerError as ex:
        status=ex.status_attributes['x-ms-status-code']
        print('There was an exception: {0}'.format(status))
        print(cosmosdb_messages[status])

In [6]:
print("Get the list of recipes that requires Egg.")
executeGremlinQuery("g.V('Egg').in('requires').hasLabel('recipe').values('id')")

Get the list of recipes that requires Egg.


['Egg Fired Rice',
 'Yang Chow Fried Rice',
 'Western Fried Rice',
 'Egg Sandwich',
 'Luncheon Meat & Egg Sandwich',
 'Tomato & Egg Sandwich']

In [11]:
print("Get the list of recipes that requires Sausage Diced.")
executeGremlinQuery("g.V('Sausage Diced').in('requires').hasLabel('recipe').values('id')")

Get the list of recipes that requires Sausage Diced.


['Western Fried Rice']

In [7]:
print("Get the list of ingredients to make Egg Fired Rice.")
executeGremlinQuery("g.V('Egg Fired Rice').out('requires').hasLabel('ingredient').values('id')")

Get the list of ingredients to make Egg Fired Rice.


['Egg', 'Cold Rice', 'Oil', 'Soy Sauce', 'Green Onion Diced', 'Salt', 'Sugar']

In [8]:
print("Get the list of requires 2 more Eggs.")
executeGremlinQuery("g.V('Egg').inE('requires').has('quantity',gte(2)).outV().values('id')")

Get the list of requires 2 more Eggs.


['Egg Fired Rice',
 'Yang Chow Fried Rice',
 'Western Fried Rice',
 'Egg Sandwich',
 'Luncheon Meat & Egg Sandwich',
 'Tomato & Egg Sandwich']

In [12]:
print("Count the total number of recipes in the Graph database.")
executeGremlinQuery("g.V().hasLabel('recipe').count()")

Count the total number of recipes in the Graph database.


[6]

In [13]:
print("Count the total number of ingredients in the Graph database.")
executeGremlinQuery("g.V().hasLabel('ingredient').count()")

Count the total number of ingredients in the Graph database.


[16]